# Utility of Wealth

## Setup

In [1]:
%pip install numpy sympy

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.


## Defining Variables and Formulas

We will define a binomial model with 5 time periods. We will call the last time period $N$ and in the code will define $N = 5$.

In [2]:
N = 5

We will define the initial stock price as $S_0 = 4$.

In [3]:
S_0 = 4

We will define the initial wealth as $X_0 = 100$.

In [4]:
X_0 = 100

The changes in stock price for a binomial model are determined by an imaginary coin flip which results in $H$ or $T$. If the result at time $n$ which we will call $\omega_n$ is $H$ the stock price at time $n-1$ will be multiplied by the up factor $u$ and if $\omega_n = T$ it will be muliplied by the down factor $d$. So, if $\omega_n = H$ then $S_n = uS_{n-1}$ and if $\omega_n = T$ then $S_n = dS_{n-1}$. We will define $u = 2$ and $d = \frac{1}{2}$.

In [5]:
u = 2
d = 1/2

We will call the probability of $H$ as $p$ and the probability of $T$ as $q$. These will be defined as $p = \frac{2}{3}$ and $q = \frac{1}{3}$.

In [6]:
p = 2/3
q = 1/3

Interest which is assumed to be equal for borrowing and holding money is defined as $r = \frac{1}{4}$.

In [7]:
r = 1/4

Risk-neutral probabilities of $H$ and $T$ which are called $\tilde{p}$ and $\tilde{q}$ respectively will be calculated using the $u$, $d$ and $r$ values. The functions `risk_neutral_p` and `risk_neutral_q` are defined below.

In [8]:
def risk_neutral_p(u, d, r):
    return (1 + r - d) / (u - d)

def risk_neutral_q(u, d, r):
    return (u - 1 - r) / (u - d)

rn_p = risk_neutral_p(u, d, r)
rn_q = risk_neutral_p(u, d, r)

The probability of event $\omega$ based on regular probabilities will be called $\mathbb{P}(\omega)$ and based on risk-neutral probabilities it will be called $\tilde{\mathbb{P}}(\omega)$. $\textrm{The Radon-Nikod}\acute{\textrm{y}} \textrm{m derivative of } \tilde{\mathbb{P}} \textrm{ with respect to } \mathbb{P}$ will thus be defined as $Z(\omega) = \frac{\tilde{\mathbb{P}}(\omega)}{\mathbb{P}(\omega)}$.

In [9]:
def calc_Z(rn_P, P):
    return rn_P / P

The state price density corresponding to $\omega$ at time $N$ will subseuqntly be defined as $\zeta(\omega) = \frac{Z(\omega)}{(1+r)^N}$.

In [10]:
def state_price_density(n, z, r):
    return z / ((1+r)**n)

## Utility

A utility function is used to capture the tradeoff between risk and return. The utility function $U$ must be a strictly concave down nondecreasing function which can take the value $-\infty$ but not the value $+\infty$. A common utility function and what we will be using for this model is $\ln(x)$. So $U(x) = \ln(x)$ and $U'(x) = \frac{1}{x}$ and the inverse of the derivative $I$ will be defuined as $I(y) = \frac{1}{y}$.

In [11]:
import numpy as np

def U(x):
    return np.log(x)

def dU(x):
    return 1/x

def I(y):
    return 1/y

The goal of this model will be to find an adapted portfolio process $\:\Delta_0,\:\Delta_1,\:.\:.\:.\:,\Delta_{N - 1}\:$, with $\Delta_n$ being the amount of stock you should own at time $n$, that maximizes $\:\mathbb{E}U(X_N)$, the expected utility of the wealth at time $N$.

## Modelling

I will start by creating a dictionary of all the possible stock prices over $5$ time periods. Next, I will create dictionaries of all the risk-neutral and regular probabilities of all the possible $H$ and $T$ combinations of length $N$. I will then get a dictionary of the $Z$ values. And, then I will make a dictionary of $\zeta$ values.

In [39]:
from itertools import permutations

def get_combinations(n):
    combinations = []
    for i in range(1, n+1):
        for j in range(i + 1):
            combo = 'H' * j + 'T' * (i - j)
            combinations += list(set([''.join(p) for p in permutations(combo)]))
    return combinations

all_combinations = get_combinations(5)

def get_S(combinations, S_0, u, d):
    price_dict = {}
    for combo in combinations:
        new_price = S_0
        for outcome in combo:
            if outcome == 'H':
                new_price *= u
            elif outcome == 'T':
                new_price *= d
        price_dict[combo] = new_price
    return price_dict

all_prices = get_S(all_combinations, S_0, u, d)

def get_P(combinations, p , q):
    prob_dict = {}
    for combo in combinations:
        prob = 1
        for outcome in combo:
            if outcome == 'H':
                prob *= p
            elif outcome == 'T':
                prob  *= q
        prob_dict[combo] = prob
    return prob_dict

P = get_P(all_combinations, p, q)
rn_P = get_P(all_combinations, rn_p, rn_q)

def get_Z(risk_neutral, normal):
    Z_dict = {}
    for key in risk_neutral.keys():
        Z_dict[key] = calc_Z(risk_neutral[key], normal[key])
    return Z_dict

Z = get_Z(rn_P, P)

def get_SPD(z_dict, r):
    spd_dict = {}
    for key, value in z_dict.items():
        spd_dict[key] = state_price_density(len(key), value, r)
    return spd_dict

SPD = get_SPD(Z, r)
SPD

{'T': 1.2,
 'H': 0.6,
 'TT': 1.44,
 'HT': 0.72,
 'TH': 0.72,
 'HH': 0.36,
 'TTT': 1.728,
 'THT': 0.864,
 'TTH': 0.864,
 'HTT': 0.864,
 'HTH': 0.432,
 'HHT': 0.432,
 'THH': 0.432,
 'HHH': 0.216,
 'TTTT': 2.0736,
 'HTTT': 1.0368,
 'TTHT': 1.0368,
 'TTTH': 1.0368,
 'THTT': 1.0368,
 'HTHT': 0.5184,
 'TTHH': 0.5184,
 'THHT': 0.5184,
 'HTTH': 0.5184,
 'THTH': 0.5184,
 'HHTT': 0.5184,
 'HTHH': 0.2592,
 'HHTH': 0.2592,
 'THHH': 0.2592,
 'HHHT': 0.2592,
 'HHHH': 0.1296,
 'TTTTT': 2.4883200000000003,
 'THTTT': 1.2441600000000002,
 'TTHTT': 1.2441600000000002,
 'HTTTT': 1.2441600000000002,
 'TTTTH': 1.2441600000000002,
 'TTTHT': 1.2441600000000002,
 'TTTHH': 0.6220800000000001,
 'HTTHT': 0.6220800000000001,
 'HTHTT': 0.6220800000000001,
 'THTTH': 0.6220800000000001,
 'TTHHT': 0.6220800000000001,
 'HTTTH': 0.6220800000000001,
 'TTHTH': 0.6220800000000001,
 'HHTTT': 0.6220800000000001,
 'THHTT': 0.6220800000000001,
 'THTHT': 0.6220800000000001,
 'HTHHT': 0.31104000000000004,
 'THTHH': 0.31104000000

$$ \textrm{For this problem we will be using Lagragian Optimization. First, we will have to define each possible portfolio value as a dummy variable.} $$
$$ x_1 = X_5(TTTTT)\;\;\;\;x_2 = X_5(TTTHT)\;\;\;\;x_3 = X_5(TTHTT)\;\;\;\;x_4 = X_5(THTTT)\;\;\;\;x_5 = X_5(HTTTT)\;\;\;\;x_6 = X_5(TTTTH)\;\;\;\;x_7 = X_5(THTTH)\;\;\;\;x_8 = X_5(THTHT)$$
$$ x_9 = X_5(THHTT)\;\;\;\;x_{10} = X_5(HTTHT)\;\;\;\;x_{11} = X_5(TTHTH)\;\;\;\;x_{12} = X_5(HTTTH)\;\;\;\;x_{13} = X_5(HHTTT)\;\;\;\;x_{14} = X_5(TTHHT)\;\;\;\;x_{15} = X_5(TTTHH)\;\;\;\;x_{16} = X_5(HTHTT)$$
$$ x_{17} = X_5(HTTHH)\;\;\;\;x_{18} = X_5(TTHHH)\;\;\;\;x_{19} = X_5(HHHTT)\;\;\;\;x_{20} = X_5(HHTTH)\;\;\;\;x_{21} = X_5(THTHH)\;\;\;\;x_{22} = X_5(HHTHT)\;\;\;\;x_{23} = X_5(THHTH)\;\;\;\;x_{24} = X_5(HTHTH)$$
$$ x_{25} = X_5(THHHT)\;\;\;\;x_{26} = X_5(HTHHT)\;\;\;\;x_{27} = X_5(HHHHT)\;\;\;\;x_{28} = X_5(HHHTH)\;\;\;\;x_{29} = X_5(THHHH)\;\;\;\;x_{30} = X_5(HHTHH)\;\;\;\;x_{31} = X_5(HTHHH)\;\;\;\;x_{32} = X_5(HHHHH)$$
$$ \textrm{We will also define the normal probabilities of the } H \textrm{ and } T \textrm{ combiantions and the } \zeta_5(\omega_1, \omega_2, \:.\:.\:.\:, \omega_5) \textrm{ values as corresponding dummy variables } p_m \textrm{ and } \zeta_m \textrm{.}$$
$$ \textrm{As said above for this model we define } U(x) = \ln x \textrm{ and our goal is to maximize } \mathbb{E}U(X_N)$$
$$ \textrm{Over } N \textrm{ time periods the number of combinations can be defined as } M = 2^N \textrm{ and in our case } M = 2^5 = 32 \textrm{. So, the problem can be redefined as maximizing } \;\displaystyle\sum_{m=1}^{32} p_mU(x_m)\; \textrm{ subject to } \;\displaystyle\sum_{m=1}^{32} p_m\zeta_mx_m = X_0 = 100\textrm{.} $$
$$ \textrm{Consequantly we can write out the Lagragian for this problem as } \;L = \displaystyle\sum_{m=1}^{32} p_mU(x_m) - \lambda(\,\displaystyle\sum_{m=1}^{32} p_m\zeta_mx_m - 100)$$
$$ \textrm{The Lagrange mulitplier equations can then be written as } \frac{\partial}{\partial x_m}L = p_mU'(x_m) - \lambda p_m\zeta_m = 0$$
$$ \textrm{This can then be simplified to } U'(x_m) = \lambda \zeta_m $$
$$ \textrm{We can then apply the inverse of } U' \textrm{ to both sides in order to isolate } x_m \textrm{ calling the inverse } I \textrm{ and from that we obtain } x_m = I(\lambda \zeta_m) $$
$$ \textrm{Next, substituting back in for the dummy variables we can write this as } X_5 = I(\lambda \zeta_5) $$
$$ \textrm{And, using our previous definition of } I(y) = \frac{1}{y} \textrm{ this can again be rewritten as } X_5 = \frac{1}{\lambda \zeta_5}. \textrm{ This also could be applied over any general value in } n = 1, 2, \:.\:.\:.\:, N \textrm{ as } X_n = \frac{1}{\lambda \zeta_n}$$
$$ \textrm{Substituting this into the formula } \mathbb{E}\zeta_N X_N = X_0 \textrm{ we obtain } \mathbb{E}[\frac{\zeta_5}{\lambda \zeta_5}] = \mathbb{E}[\frac{1}{\lambda}] = \frac{1}{\lambda} = X_0 = 100 \;\textrm{ so }\; \lambda = \frac{1}{X_0} = \frac{1}{100} $$

In [42]:
lmbda = 1/X_0

def calc_weath(inverse, lmbda, spd):
    return inverse(lmbda * spd)

def get_wealth(inverse, lmbda, spd_dict):
    wealth_dict = {}
    for key, value in spd_dict.items():
        wealth_dict[key] = calc_weath(inverse, lmbda, value)
    return wealth_dict
        
wealth = get_wealth(I, lmbda, SPD)
wealth

{'T': 83.33333333333333,
 'H': 166.66666666666666,
 'TT': 69.44444444444444,
 'HT': 138.88888888888889,
 'TH': 138.88888888888889,
 'HH': 277.77777777777777,
 'TTT': 57.87037037037037,
 'THT': 115.74074074074073,
 'TTH': 115.74074074074073,
 'HTT': 115.74074074074073,
 'HTH': 231.48148148148147,
 'HHT': 231.48148148148147,
 'THH': 231.48148148148147,
 'HHH': 462.96296296296293,
 'TTTT': 48.2253086419753,
 'HTTT': 96.4506172839506,
 'TTHT': 96.4506172839506,
 'TTTH': 96.4506172839506,
 'THTT': 96.4506172839506,
 'HTHT': 192.9012345679012,
 'TTHH': 192.9012345679012,
 'THHT': 192.9012345679012,
 'HTTH': 192.9012345679012,
 'THTH': 192.9012345679012,
 'HHTT': 192.9012345679012,
 'HTHH': 385.8024691358024,
 'HHTH': 385.8024691358024,
 'THHH': 385.8024691358024,
 'HHHT': 385.8024691358024,
 'HHHH': 771.6049382716049,
 'TTTTT': 40.187757201646086,
 'THTTT': 80.37551440329217,
 'TTHTT': 80.37551440329217,
 'HTTTT': 80.37551440329217,
 'TTTTH': 80.37551440329217,
 'TTTHT': 80.37551440329217,
 

$$ \textrm{Lastly, to calculate the adapted portfolio processes we can use the recursive formula } \Delta_{n - 1}(\omega_1, \omega_2, \:.\:.\:.\:, \omega_{n-1}) = \frac{X_n(\omega_1, \omega_2, \:.\:.\:.\:, \omega_{n-1},H) - X_n(\omega_1, \omega_2, \:.\:.\:.\:, \omega_{n-1},T)}{S_n(\omega_1, \omega_2, \:.\:.\:.\:, \omega_{n-1},H) - S_n(\omega_1, \omega_2, \:.\:.\:.\:, \omega_{n-1},T)}$$

In [48]:
def calc_delta(x_H, x_T, s_H, s_T):
    return (x_H - x_T) / (s_H - s_T)

def get_optimal_process(price_dict, wealth_dict):
    process_dict = {}
    process_dict['0'] = calc_delta(wealth_dict['H'], wealth_dict['T'], price_dict['H'], price_dict['T'])
    for key in price_dict.keys():
        if len(key) >= 5:
            return process_dict
        process_dict[key] = calc_delta(wealth_dict[key + 'H'], wealth_dict[key + 'T'], price_dict[key + 'H'], price_dict[key + 'T'])

processes = get_optimal_process(all_prices, wealth)

# Print everything neatly 
output_str = ""
count = 0
for key, value in processes.items():
    count += 1
    output_str += f"∆({key}) = {round(value, 3)}  "
    if count % 10 == 0 and key != "HHHT":
        print(output_str)
        output_str = ""
print(output_str)

∆(0) = 13.889  ∆(T) = 23.148  ∆(H) = 11.574  ∆(TT) = 38.58  ∆(HT) = 19.29  ∆(TH) = 19.29  ∆(HH) = 9.645  ∆(TTT) = 64.3  ∆(THT) = 32.15  ∆(TTH) = 32.15  
∆(HTT) = 32.15  ∆(HTH) = 16.075  ∆(HHT) = 16.075  ∆(THH) = 16.075  ∆(HHH) = 8.038  ∆(TTTT) = 107.167  ∆(HTTT) = 53.584  ∆(TTHT) = 53.584  ∆(TTTH) = 53.584  ∆(THTT) = 53.584  
∆(HTHT) = 26.792  ∆(TTHH) = 26.792  ∆(THHT) = 26.792  ∆(HTTH) = 26.792  ∆(THTH) = 26.792  ∆(HHTT) = 26.792  ∆(HTHH) = 13.396  ∆(HHTH) = 13.396  ∆(THHH) = 13.396  ∆(HHHT) = 13.396  ∆(HHHH) = 6.698  
